In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, DoubleType, IntegerType
from pyspark.sql.functions import sum as _sum, avg, expr, window, from_unixtime, col

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
import time, datetime
import os
import random
import uuid

spark = (
        SparkSession.builder.appName("spark application")
        .config(
            "spark.jars.packages",
            "org.apache.hudi:hudi-spark3.1-bundle_2.12:0.13.0",
        )
        .config(
            "spark.sql.extensions",
            "org.apache.spark.sql.hudi.HoodieSparkSessionExtension",
        )
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
        .config(
            "spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem"
        )
        # .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
        # .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
        # .config("spark.hadoop.fs.s3a.endpoint", "http://10.159.37.34:9000")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
        .config(
            "spark.hadoop.fs.s3a.aws.credentials.provider",
            "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider",
        )
        .config("spark.hadoop.fs.s3a.connection.maximum", "1000")
        .getOrCreate()
    )

23/06/20 10:15:37 WARN Utils: Your hostname, ducdn resolves to a loopback address: 127.0.1.1; using 10.1.124.58 instead (on interface enp1s0)
23/06/20 10:15:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ducdn/Documents/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ducdn/.ivy2/cache
The jars for the packages stored in: /home/ducdn/.ivy2/jars
org.apache.hudi#hudi-spark3.1-bundle_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a6665dc5-8363-4e01-8df5-02b1eb6d9725;1.0
	confs: [default]
	found org.apache.hudi#hudi-spark3.1-bundle_2.12;0.13.0 in central
:: resolution report :: resolve 321ms :: artifacts dl 12ms
	:: modules in use:
	org.apache.hudi#hudi-spark3.1-bundle_2.12;0.13.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent

In [2]:
hudi_path = "/home/ducdn/Desktop/workspace/hudi_mor/hudi_mor"

#### Snapshot

In [3]:
df = spark.read.format("hudi").load(hudi_path)
df.show()

23/06/20 10:17:11 WARN DFSPropertiesConfiguration: Cannot find HUDI_CONF_DIR, please set it as the dir of hudi-defaults.conf
23/06/20 10:17:11 WARN DFSPropertiesConfiguration: Properties file file:/etc/hudi/conf/hudi-defaults.conf not found. Ignoring to load props file


+-------------------+--------------------+--------------------+----------------------+--------------------+--------------------+----------+----------+---------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|                  id|     value| timestamp|partition|
+-------------------+--------------------+--------------------+----------------------+--------------------+--------------------+----------+----------+---------+
|  20230620100055605|20230620100055605...|be851e8f-1ee1-435...|                     2|6ef76c77-4382-4f7...|be851e8f-1ee1-435...|    195634|1687230054|        2|
|  20230620100109116|20230620100109116...|9ee8972b-91ad-433...|                     2|6ef76c77-4382-4f7...|9ee8972b-91ad-433...|    577525|1687230068|        2|
|  20230620100544485|20230620100544485...|965e97b6-c5d7-47b...|                     2|6ef76c77-4382-4f7...|965e97b6-c5d7-47b...|  11223344|1687230344|        2|
|  20230620100234418|2023062010023

In [4]:
df.createOrReplaceTempView("hudi_sn")
spark.sql("select * from hudi_sn order by _hoodie_commit_time").show()

+-------------------+--------------------+--------------------+----------------------+--------------------+--------------------+----------+----------+---------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|                  id|     value| timestamp|partition|
+-------------------+--------------------+--------------------+----------------------+--------------------+--------------------+----------+----------+---------+
|  20230620100055605|20230620100055605...|8989e5b9-2ca5-4a8...|                     1|c0efbf50-d9b2-417...|8989e5b9-2ca5-4a8...|    767664|1687230054|        1|
|  20230620100055605|20230620100055605...|be851e8f-1ee1-435...|                     2|6ef76c77-4382-4f7...|be851e8f-1ee1-435...|    195634|1687230054|        2|
|  20230620100055605|20230620100055605...|2d03e799-af19-448...|                     3|076ad2ab-de6e-4ea...|2d03e799-af19-448...|    754587|1687230054|        3|
|  20230620100109116|2023062010010

#### Incremental query

In [17]:
# Cấu hình truy vấn incremental
incrementalOptions = {
    "hoodie.datasource.query.type": "incremental",
    "hoodie.datasource.read.begin.instanttime": "20230620100055605",
    # "hoodie.datasource.read.end.instanttime": "20230620100109120"
    # "hoodie.datasource.read.partition.path": "/year=2020/month=*/day=*"  # Tuỳ chọn, sử dụng glob pattern nếu truy vấn các phân vùng cụ thể
}

hudiIncQueryDF = spark.read \
    .format("org.apache.hudi") \
    .options(**incrementalOptions) \
    .load(hudi_path)

hudiIncQueryDF.createOrReplaceTempView("hudi_incre")
spark.sql("select * from hudi_incre order by _hoodie_commit_time").show(truncate=False)

+-------------------+---------------------+------------------------------------+----------------------+----------------------------------------------------------------------------+------------------------------------+----------+----------+---------+
|_hoodie_commit_time|_hoodie_commit_seqno |_hoodie_record_key                  |_hoodie_partition_path|_hoodie_file_name                                                           |id                                  |value     |timestamp |partition|
+-------------------+---------------------+------------------------------------+----------------------+----------------------------------------------------------------------------+------------------------------------+----------+----------+---------+
|20230620100109116  |20230620100109116_1_1|9ee8972b-91ad-4331-bb82-0138f1187bcc|2                     |6ef76c77-4382-4f70-9891-641bfeb27131-0_0-1358-2753_20230620101158263.parquet|9ee8972b-91ad-4331-bb82-0138f1187bcc|577525    |1687230068|2        |


#### Timetravel query

In [21]:
df = spark.read.format("hudi").option("as.of.instant","20230620100055605").load(hudi_path)
df.show()

+-------------------+--------------------+--------------------+----------------------+--------------------+--------------------+------+----------+---------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|                  id| value| timestamp|partition|
+-------------------+--------------------+--------------------+----------------------+--------------------+--------------------+------+----------+---------+
|  20230620100055605|20230620100055605...|be851e8f-1ee1-435...|                     2|6ef76c77-4382-4f7...|be851e8f-1ee1-435...|195634|1687230054|        2|
|  20230620100055605|20230620100055605...|2d03e799-af19-448...|                     3|076ad2ab-de6e-4ea...|2d03e799-af19-448...|754587|1687230054|        3|
+-------------------+--------------------+--------------------+----------------------+--------------------+--------------------+------+----------+---------+



#### Snapshot after cleaning

In [19]:
df = spark.read.format("hudi").load(hudi_path)
df.createOrReplaceTempView("hudi_sn")
spark.sql("select * from hudi_sn order by _hoodie_commit_time").show(n=100)

+-------------------+--------------------+--------------------+----------------------+--------------------+--------------------+----------+----------+---------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|                  id|     value| timestamp|partition|
+-------------------+--------------------+--------------------+----------------------+--------------------+--------------------+----------+----------+---------+
|  20230620100055605|20230620100055605...|be851e8f-1ee1-435...|                     2|6ef76c77-4382-4f7...|be851e8f-1ee1-435...|    195634|1687230054|        2|
|  20230620100055605|20230620100055605...|2d03e799-af19-448...|                     3|076ad2ab-de6e-4ea...|2d03e799-af19-448...|    754587|1687230054|        3|
|  20230620100055605|20230620100055605...|8989e5b9-2ca5-4a8...|                     1|c0efbf50-d9b2-417...|8989e5b9-2ca5-4a8...|    767664|1687230054|        1|
|  20230620100109116|2023062010010